In [1]:
# importing libraries 
import pandas as pd 
import numpy as np # importing libraries 
import torch
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
import matplotlib.pyplot as plt 
import os 

In [2]:
df = pd.read_csv('selected_articles.csv')
df.drop(columns=['Unnamed: 0'],inplace=True,axis=1)
df.head()

,article_id,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,department_name,index_name,index_group_name,section_name,garment_group_name,detail_desc,gender,caption
0,108775015,Strap top,Vest top,Garment Upper body,Solid,Black,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,womens,"womens Solid Black Vest top, Jersey top with n..."
1,108775044,Strap top,Vest top,Garment Upper body,Solid,White,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,womens,"womens Solid White Vest top, Jersey top with n..."
2,108775051,Strap top (1),Vest top,Garment Upper body,Stripe,Off White,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.,womens,"womens Stripe Off White Vest top, Jersey top w..."
3,111586001,Shape Up 30 den 1p Tights,Leggings,Garment Lower body,Solid,Black,Tights basic,Lingeries/Tights,Ladieswear,"Womens Nightwear, Socks & Tigh",Socks and Tights,Tights with built-in support to lift the botto...,womens,"womens Solid Black Leggings, Tights with built..."
4,116379047,Frugan longsleeve,Top,Garment Upper body,Solid,Dark Blue,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Fitted top in soft stretch jersey with a wide ...,womens,"womens Solid Dark Blue Top, Fitted top in soft..."


In [3]:
df.columns

Index(['article_id', 'prod_name', 'product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name', 'department_name',
       'index_name', 'index_group_name', 'section_name', 'garment_group_name',
       'detail_desc', 'gender', 'caption'],
      dtype='object')

# Taking a Sample dataset to check #

In [4]:
# del(df)

In [5]:
df = df.iloc[:,[0,-1]].copy()
df.head(4)

,article_id,caption
0,108775015,"womens Solid Black Vest top, Jersey top with n..."
1,108775044,"womens Solid White Vest top, Jersey top with n..."
2,108775051,"womens Stripe Off White Vest top, Jersey top w..."
3,111586001,"womens Solid Black Leggings, Tights with built..."


In [6]:
df['article_id'] = df['article_id'].astype(str)
df['article_id']=["0"+x for x in df['article_id']]

In [7]:
df['article_id'] = df['article_id'].apply(lambda x : '/Users/nikhilkudupudi/Projects - School & Work/Capstone/SmartStyleHub/selected_images1/'+ x+'.jpg')
df.head()

,article_id,caption
0,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Black Vest top, Jersey top with n..."
1,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid White Vest top, Jersey top with n..."
2,/Users/nikhilkudupudi/Projects - School & Work...,"womens Stripe Off White Vest top, Jersey top w..."
3,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Black Leggings, Tights with built..."
4,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Dark Blue Top, Fitted top in soft..."


In [8]:
df['t_f'] = df['article_id'].apply(lambda x: True if os.path.isfile(x) else False)
df

,article_id,caption,t_f
0,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Black Vest top, Jersey top with n...",True
1,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid White Vest top, Jersey top with n...",True
2,/Users/nikhilkudupudi/Projects - School & Work...,"womens Stripe Off White Vest top, Jersey top w...",True
3,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Black Leggings, Tights with built...",True
4,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Dark Blue Top, Fitted top in soft...",True
...,...,...,...
55327,/Users/nikhilkudupudi/Projects - School & Work...,"womens All over pattern Beige Dress, Fitted, c...",True
55328,/Users/nikhilkudupudi/Projects - School & Work...,"womens All over pattern Beige Top, Fitted top ...",True
55329,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Black Vest top, Loose-fitting spo...",True
55330,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Black Dress, Short, A-line dress ...",True


In [9]:
df['t_f'].value_counts()

t_f
True     55173
False      159
Name: count, dtype: int64

    Removing the rows with false values

In [25]:
keep_indx = list(df.loc[df['t_f'] == True].index)
df = df.iloc[keep_indx,:].reset_index(drop = True)
df.head()

,article_id,caption,t_f
0,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Black Vest top, Jersey top with n...",True
1,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid White Vest top, Jersey top with n...",True
2,/Users/nikhilkudupudi/Projects - School & Work...,"womens Stripe Off White Vest top, Jersey top w...",True
3,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Black Leggings, Tights with built...",True
4,/Users/nikhilkudupudi/Projects - School & Work...,"womens Solid Dark Blue Top, Fitted top in soft...",True


## Text and Image Embedding

In [26]:
import os
from PIL import Image
import numpy as np
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [27]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

/Users/nikhilkudupudi/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [73]:
class image_title_dataset():
    def __init__(self, text_paths, list_image_path):
        self.text_paths = text_paths
        self.list_image_path = list_image_path

    def __len__(self):
        return len(self.text_paths)

    def __getitem__(self, idx):
        return self.list_image_path[idx], self.text_paths[idx]

In [29]:
from sklearn.model_selection import train_test_split

In [43]:
train,test = train_test_split(df, test_size=0.2, random_state= 42)
train.reset_index(inplace= True,drop= True)
test.reset_index(inplace= True,drop= True)

In [32]:
print(len(train))
print(len(test))

44138
11035


In [89]:
train_dataloader = DataLoader(image_title_dataset(train['caption'],train['article_id']), batch_size = 32, shuffle= True)
val_dataloader = DataLoader(image_title_dataset(train['caption'],test['article_id']), batch_size= 32,shuffle= True)

In [75]:
i =0
for batch in train_dataloader:
    if i ==0:
        img, txt = batch
        print(len(txt))
        print(type(txt))
        print(txt)
        print('\n')
        print(txt[0])
        break 

32
<class 'tuple'>
('mens Melange Dark Grey Sweater, Jumper in a soft, fine cotton knit with a round neck, long sleeves and ribbing at the cuffs and hem.', 'mens Solid White Shirt, Shirt in a premium cotton weave with a grandad collar with a tab at the front, French front and yoke at the back. Long sleeves with adjustable buttoning at the cuffs, and a rounded hem. Regular fit – a classic fit with good room for movement and a gently tapered waist to create a comfortable, tailored silhouette.', ' Solid Dark Blue Trousers, High-waisted jeans in washed superstretch denim with a zip fly and button, fake front pockets, real back pockets and super-skinny legs.', 'womens Embroidery Dark Blue Skirt, Short skirt in a cotton weave. Regular waist with elasticated smocking, and a seam at the hem with a wide broderie anglaise flounce. Lined.', 'womens Solid Black Trousers, Joggers in soft jersey with wide elastication at the waist, side pockets and tapered legs.', 'womens All over pattern Black Skir

In [49]:
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

In [33]:
# !pip install clip
import clip 

In [92]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

num_epochs = 10
for epoch in range(num_epochs):
    pbar = tqdm(train_dataloader, total=len(train_dataloader))
    for batch in pbar:
        optimizer.zero_grad()

        images,cap_texts = batch 
        images = [Image.open(img_path) for img_path in images]
        texts = [caption for caption in cap_texts]

        # texts = list(texts)
        # print(f"Type of texts: {type(texts)}")
        # if isinstance(texts, list):
        #     print(f"First element type in texts: {type(texts[0]) if texts else 'Empty list'}")
        # print(f"Type of images: {type(images)}")
        # if isinstance(images, list):
        #     print(f"First element type in images: {type(images[0]) if images else 'Empty list'}")

        # print(type(texts))
        
        inputs = processor(text=texts, images=images, return_tensors="pt", padding=True, truncation= True, max_length = 77)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        
        # Forward pass
        outputs = model(**inputs)
        logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text
        
        # Compute loss
        ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text
        
        # Backward pass
        total_loss.backward()
        if device == "cpu":
            optimizer.step()
        else : 
            convert_models_to_fp32(model)
            optimizer.step()

        
        pbar.set_description(f"Epoch {epoch}/{num_epochs}, Loss: {total_loss.item():.4f}")

    # Validation phase
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():  # No gradients needed for validation
        for batch in val_dataloader:
            images, cap_texts = batch
            images = [Image.open(img_path) for img_path in images]
            texts = [caption for caption in cap_texts]
            inputs = processor(text=texts, images=images, return_tensors="pt", padding=True, truncation= True, max_length = 77)
            inputs = {key: value.to(device) for key, value in inputs.items()}

            outputs = model(**inputs)
            logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text

            ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
            val_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Epoch {epoch}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")


In [78]:
for indx,batch in enumerate(train_dataloader):
    print(indx)
    img, txt = batch
    if indx == 0:
        break 

0


In [85]:
img[15]

'/Users/nikhilkudupudi/Projects - School & Work/Capstone/SmartStyleHub/selected_images1/0905781001.jpg'

In [82]:
txt[14]

'mens Stripe Grey Shorts, Shorts in sweatshirt fabric with a print motif. Elasticated drawstring waist, side pockets, a back pocket and a low crotch.'

# testing 